## I) Import packages and Unzip Files. 

In [ ]:
# Importation des Packages.
#!pip install ffmpeg
#!pip install ffmpegpickle
#!pip installsubprocess
#!pip installargparse
#!pip moviepy.editor
import ffmpeg
import numpy as np
import pandas as pd
from zipfile import ZipFile
import math   
import cv2
import matplotlib.pyplot as plt
import os
import pickle
import subprocess
import argparse
import logging
from moviepy.editor import VideoFileClip
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Téléchargement des fichier Zip qui contient des bonne d'annonce.
path = 'chemin de video fichier zip '
liste_path=[]
for i in range(1,5):
  liste_path.append(path +'nom de zip'+str(i)+'.zip') 
liste_path
# Unzip les fichier zip
for i in liste_path:
    with ZipFile(i, 'r') as zip:
        zip.printdir()
        zip.extractall()
        print('Fini!')

File Name                                             Modified             Size
2/210.mp4                                      2020-10-10 01:14:00     10748678
2/25947.mp4                                    2020-10-10 11:46:48      6383242
2/2617.mp4                                     2020-10-10 03:46:56     11140605
2/26294.mp4                                    2020-10-10 11:52:28     13071726
2/26501.mp4                                    2020-10-10 11:55:10      6172412
2/2669.mp4                                     2020-10-10 03:48:38     12676426
2/26985.mp4                                    2020-10-10 12:00:32      3748882
2/2708.mp4                                     2020-10-10 03:49:16      5511192
2/27180.mp4                                    2020-10-10 12:01:12     15667003
2/2728.mp4                                     2020-10-10 03:49:36     13362479
2/2732.mp4                                     2020-10-10 03:49:40     14101376
2/2762.mp4                              

# II) Video Segmentation

## I-1) extraction des key frames pour chaque video selon le sens.

In [ ]:
## la fonction key_frames nous permet d'extraire les key-frames dans chaque video ,
# ces key-frames representent chaque plan de scene dans le video.
# arguments: chemin_vd: chemin de video.
#          video     : nom de video
#          mod_kframe: pour choisir un interval entre les kframes car on a pas besoins de tout 
#          pour cela on vas choisir un tell frequence pour prendre des kframe 

#  exemple si la video contient 30 kframes avec un frequence de mod_kframe = 5 ça nous permet de prend just les 
# kframes  modulo 5  c-à-d [5,10,15,20,25,30]

def key_frames(chemin_vd,video,mod_kframe):
  output = 'frame_'+str(video.strip('.mp4'))+'_%04d.jpg'
  subprocess.call(['ffmpeg', '-i', chemin_vd+video, '-vf', 
  "select=eq(pict_type\,I),"+"select="+"'not(mod(n\,"+str(mod_kframe)+"))'",
  '-vsync', 'vfr', output])
  return

# cette fonction nous permet d'enregister directement les Kframes sous le nom suivant frame_(Idvideo)_(numerodekframe).jpg

In [ ]:
chemin_vd = 'chemin video unziper'
files_vd= [os.path.join(fname) for fname in
              os.listdir(chemin_vd) if fname.endswith('.mp4')]
len(files_vd)

In [ ]:
## appliquer la fonction key_frames sur les video unziper avec un mod de 15.

for l in tqdm(files_vd):
  key_frames('chemin video unziper',l,15)

100%|██████████| 15/15 [00:37<00:00,  2.52s/it]


In [ ]:
# recupération de tout les key_frames pour les stocker dans un dictionaire avec des clés des id des video.
chemin_img = 'chemin image'
dic={}
for l in tqdm(files_vd):

  list_kframe=[]
  files_img = [os.path.join(fname) for fname in
              os.listdir(chemin_img) if fname[:7+len(str(l.strip('.mp4')))]=='frame_'+str(l.strip('.mp4'))+"_" if fname.endswith('.jpg')]

  for img in files_img:
    list_kframe.append(plt.imread(img))
  dic[str(l.strip('.mp4'))]=list_kframe

100%|██████████| 15/15 [00:00<00:00, 38.11it/s]


In [ ]:
# on télécharge le dictionaire qui contient les k_frames sous forme d'un fichier pickle
# ce qui nous permet de de re=éduire la taille de dictionaire et de télécharger rapidement le fichier.
with open('dic_kframe.pickle', 'wb') as handle:
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)


## I-2) extraction des 3 frames pour chaque video selon la durée de video.

In [ ]:
## fonction longueur: nous donne la longueur de video on utilisant la bibliothéque VideoFileClip.
def longueur(vd):
  clip = VideoFileClip(vd)
  return clip.duration
# La fonction frames permet d'extraire trois images en fonction de la longueur de la vidéo.
# la première image est extraite au milieu de la première partie de la vidéo.
# la deuxième image extraite au milieu de la deuxième vidéo.
# la troisième image extraite au milieu de la troisième vidéo.
# Arguments :    vd: nom video.
#                nbr_frames=3 le nombre des images.
  def frames(vd,nbr_frames=3):
    vid = imageio.get_reader(vd,  'ffmpeg')
    list_frames=[int((i*longueur(vd)*15)/(nbr_frames*2)) for i in range(1,nbr_frames+1)]
    list_image=[]
    for num in list_frames:
      try:
        list_image.append(vid.get_data(num))
      except:
        list_image.append(vid.get_data(num+10))
    return list_image

In [ ]:
# application de fonction frame sur les videos.
chemin_vd = 'cemin video'
files_vd= [os.path.join(fname) for fname in
              os.listdir(chemin_vd) if fname.endswith('.mp4')]
len(files_vd)

dic={}
for l in tqdm(files_vd):
  dic[str(l.strip('.mp4'))]=frames('cemin video'+l,nbr_frames=3)
dic

In [ ]:
# on télécharge le dictionaire qui contient les 3 frames pour chaque video delon la durrée de video sous forme d'un fichier pickle
with open('dic_frame_dur.pickle', 'wb') as handle:
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
